Para desarrollar el método del punto medio requiere:
- La función de la forma $ \frac{dy}{dt} = f(t, y) $ es decir tenemos una función con dos variables independientes.
- Nos puede dar el *h* o el intervalo y el número de nodos requeridos *n*, si es esto 
ultimo calculamos $ h = \frac{x_f -x_i}{n} $
- A partir del intervalo y *h* calculamos los nodos, es decir los valores que se someteran al cálculo
- El primer valor $y_0$ será proporcionado como la evaluación de la función en el inicio del intervalo y a partir de este calculamos el resto de valores con las fórmula: 
$$ y_i = y_{i-1} + hk_2  $$
En donde:
$$ k_1 = f(t_{i-1}, y_{i-1}) $$
$$ k_2 = f(t_{i-1} + \frac{1}{2} h, y_{i-1} + ) \frac{1}{2} h k_1 $$

Ejemplo de clase 15.2.2 :udbvirtual.edu.sv/materiales_didacticos/AMN941/clase15.html

Emplear el método del punto medio para obtener la solución aproximada de la siguiente ecuación diferencial:

$$ (e^{2y} - y cos(ty))dt + (2te^{2y} + 2y - tcos(ty))dy=0, \ \ \ \frac{\pi}{2} \leq t \leq \frac{801 \pi}{1600}, \ \ \ y(\pi / 2) = 1, \ \ \ h= \pi / 6400 $$

Primero la despejamos para dejar la forma: $ \frac{dy}{dt} = f(t, y) $ y obtenemos

$$ \frac{dy}{dt}= \frac{e^{2y} - y cos(ty)}{tcos(ty) - 2te^{2y} - 2y } $$

Pasamos a la solución en python

In [1]:
from sympy import *
from numpy import arange

fy = symbols("fy", cls=Function)
t, y = symbols("t y")

h = pi/6400

eq = Eq(fy(t).diff(), (exp(2*fy(t)) - fy(t)*cos(t*fy(t))) / (t*cos(t*fy(t)) - 2*t*exp(2*fy(t)) - 2*fy(t)))
f = (exp(2*y) - y*cos(t*y)) / (t*cos(t*y) - 2*t*exp(2*y)-2*y)


# Calculamos la solucion particular F
condiciones_iniciales = {fy(pi/2):1}
#F = dsolve(eq, ics=condiciones_iniciales, hint='lie_group',)


# creamos nuestro intervalo
T = [t for t in arange(pi/2, 801*pi /1600 + h/2, h)]
T

[pi/2, 3201*pi/6400, 1601*pi/3200, 3203*pi/6400, 801*pi/1600]

In [5]:
# Declaro la lista con y_0 = 0

Y = [1]
K1 = [0]
K2 = [0]

# Hago un for para aplicar la fórmula
for i in range(1, len(T)):
    k1 = f.evalf(subs={t: T[i-1], y: Y[i-1]})
    k2 = f.evalf(subs={t: T[i-1] + 0.5*h, y: Y[i-1] + 0.5*h*k1})
    K1.append(k1)
    K2.append(k2)
    Y.append(float(Y[i-1] + h * k2))

Y

[1,
 0.9998561643217644,
 0.9997123688671469,
 0.9995686136125013,
 0.9994248985342039]

In [3]:
# como no fue posible resolver la EDI de forma simbólica, lo hacemos de forma 
# númerica con scipy
from scipy.integrate import solve_ivp

# hay que llamar a las funciones trigonométricas y a los e desde numpy
import numpy

def funcionEq(t, y): 
    return (numpy.exp(2*y) - y*numpy.cos(t*y)) / (t*numpy.cos(t*y) - 2*t*numpy.exp(2*y)-2*y)

solucion = solve_ivp(funcionEq, (float(pi/2), float(801*pi /1600)), [1], t_eval=T)
solucion_exacta = []
for i in solucion.y[0]:
    solucion_exacta.append(float(i))
solucion_exacta

[1.0,
 0.9998561643206377,
 0.9997123688648947,
 0.9995686136091246,
 0.999424898529704]

In [7]:
# pasamos todos los datos a una lista anidada para poder desplegarlos en una tabla

from utils import imprimir_tabla

lista_tabla = [["ti", "k1", "k2", "Yi", "F(ti)", "Error"]]

for i in range(len(T)):
    lista_tabla.append(
        [
            str(float(T[i])),
            str(K1[i]),
            str(K2[i]),
            str(float(Y[i])),
            str(solucion_exacta[i]),
            str(abs(Y[i] - solucion_exacta[i])),
        ]
    )

imprimir_tabla(lista_tabla)

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ ti               ┃ k1               ┃ k2               ┃ Yi               ┃ F(ti)            ┃ Error            ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ 1.57079632679489 │ 0                │ 0                │ 1.0              │ 1.0              │ 0.0              │
│ 66               │                  │                  │                  │                  │                  │
│ 1.57128720064702 │ -0.2930606273631 │ -0.2930196375574 │ 0.99985616432176 │ 0.99985616432063 │ 1.12665432538960 │
│                  │ 88               │ 95               │ 44               │ 77               │ 89e-12           │
│ 1.57177807449914 │ -0.2929786603781 │ -0.2929376946755 │ 0.99971236886714 │ 0.99971236886489 │ 2.25219842775459 │
│ 34               │ 99               │ 96               │ 69               │ 47               │ 26e-12           │
│ 1.57226894835126 │ -0.2928967415875 │ -0.2928557999651 │ 0.99956861361250 │ 0.99956861360912 │ 3.37663230709495 │
│ 68               │ 00               │ 91               │ 13               │ 46               │ 1e-12            │
│ 1.57275982220339 │ -0.2928148709455 │ -0.2927739533807 │ 0.99942489853420 │ 0.99942489852970 │ 4.49995596341068 │
│ 03               │ 14               │ 29               │ 39               │ 4                │ 45e-12           │
└──────────────────┴──────────────────┴──────────────────┴──────────────────┴──────────────────┴──────────────────┘